# Travaux Pratiques d'imagerie par rayons X
NB : Vous pouvez travailler en groupe mais le travail est à rendre individuellement.

In [1]:
import numpy as np
import xraylib as xrl
from xpecgen import xpecgen as xg
import ipywidgets as widgets
from tabletext import to_text
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

# Loi d'atténuation
Le but est de retrouver le coefficient d’atténuation à 100 keV de matériaux à travers une simulation réaliste en utilisant la loi de Beer-Lambert :
$$
N_{\mathrm{dt}}=N_0\exp(-\mu(E)X)
$$
où $N_0$ est le nombre de photons incident sur l'objet d'épaisseur $X$ et $\mu$ le coefficient d'atténuation linéique à l'énergie $E$.

## Conditions expérimentales
Le code suivant simule un nombre de photons détectés après avoir traversé 0.1 mm de plomb (Pb, densité 11.35 g.cm$^{⁻3}$) et 3.9 cm de polyéthylène (H$_2$C, densité 0.94 g.cm$^{⁻3}$) étant donné un nombre de photons incidents. Le processus stochastique d'atténuation est reproduit (par une loi de probabilité de Poisson).

## Travail demandé
Retrouvez le coefficient d'atténuation du plomb et du polyéthylène à 100 keV avec une précision de $10^{-3}$. Il vous faudra adapter le nombre de photons envoyés. Pour mémoire, la loi de propagation d'erreur d'une fonction $y(x_1, ..., x_N)$ est
$$
\mathrm{Var}[y]=\sum_{i=1}^N\left(\dfrac{\partial y}{\partial x_i}\right)^2\mathrm{Var}[x_i]
$$
où la variance $\mathrm{Var}[]$ représente le carré de l'erreur (ou écart-type).

Vérifiez la concordance du résultat obtenu avec le coefficient d’atténuation linéique théorique obtenu sur la base de données [XCOM](http://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html) de [NIST](http://www.nist.gov/pml/data/xray_gammaray.cfm).

In [2]:
def simulator(N0):
    dH2C = xrl.GetCompoundDataNISTByName('Polyethylene')['density']
    NdtH2C = np.random.poisson(N0*np.exp(-3.9*dH2C*xrl.CS_Total_CP('H2C', 100)))
    print('%d photons ont traversé la plaque de 3.9 cm de polyéthylène sur %d envoyés' % (NdtH2C,N0))
    dPb = xrl.ElementDensity(82)
    NdtPb = np.random.poisson(N0*np.exp(-0.01*dPb*xrl.CS_Total(82, 100)))
    print('%d photons ont traversé la plaque de 0.1 mm de plomb sur %d envoyés' % (NdtPb,N0))

simulator(100)

50 photons ont traversé la plaque de 3.9 cm de polyéthylène sur 100 envoyés
49 photons ont traversé la plaque de 0.1 mm de plomb sur 100 envoyés


# Étude du polychromatisme

La largeur du spectre en énergie des photons incidents a une forte influence sur la formation de
l’image radiologique. Nous l’étudions ici.

## Conditions expérimentales. 

Le logiciel de simulation ci-dessous permet de calculer des spectres tels que générés par les générateurs de rayons X en précisant :
- la haute tension U d’accélération des électrons
- l'épaisseur d'eau 

Certaines grandeurs associées aux spectres sont calculées en sortie notamment
- la couche de demi-atténuation (en mm d’Al),
- l’énergie moyenne des photons du spectre

## Travail demandé. 

Dans le but d’observer l’évolution du spectre en fonction de l’épaisseur d’eau
traversée (phénomène de durcissement de faisceau), tracer l’évolution de l’énergie moyenne et de la
couche de demi-atténuation en fonction de l’épaisseur d’eau (de 0 mm à 500 mm).

### NB : caractéristiques du tube X
- réglé pour avoir 1Gy à 1m dans l'air en absence objet
- filtration interne de 1.2mm d'aluminium


In [5]:
def spectrum(E0,H2O_thickness):
    #Calculate a spectrum
    xray_spectrum=xg.calculate_spectrum(E0,12,3,100,epsrel=0.5,monitor=None)
    #Inherent filtration
    xray_spectrum.attenuate(0.12,xg.get_mu(13)) #1.2 mm of Al
    xray_spectrum.attenuate(100,xg.get_mu("air")) #100 cm of Air
    #Normalize the spectrum in the sense of dose
    fluence_to_dose=xg.get_fluence_to_dose()
    xray_spectrum.set_norm(value=1,weight=fluence_to_dose)
    #Water attenuation    
    H2O_mass = 2. * xrl.AtomicWeight(1) + xrl.AtomicWeight(8)
    H_weight = H2O_thickness * 2. * xrl.AtomicWeight(1) / (xrl.ElementDensity(1) * H2O_mass)
    O_weight = H2O_thickness * xrl.AtomicWeight(8) / (xrl.ElementDensity(8) * H2O_mass)
    xray_spectrum.attenuate(H_weight,xg.get_mu(1))
    xray_spectrum.attenuate(O_weight,xg.get_mu(8))
    #Get the figures
    Nr_Photons = "%.4g" % (xray_spectrum.get_norm())
    Average_Energy = "%.2f keV" % (xray_spectrum.get_norm(lambda x:x)/xray_spectrum.get_norm())
    Dose = "%.4g mGy" % (xray_spectrum.get_norm(fluence_to_dose))
    #Calculate HVL
    mu_Al=xg.get_mu(13)
    HVL_Al=xray_spectrum.hvl(0.5,fluence_to_dose,mu_Al)
    HVL_Al_text = "%.2f mm Al" % (10*HVL_Al)
    a = [["Dose à 1m", Dose],["Nombre total de photons", Nr_Photons],["Énergie moyenne des photons",Average_Energy],["Couche de Demie-Atténuation", HVL_Al_text]]
    print(to_text(a))
    (x2,y2) = xray_spectrum.get_points()
    plt.figure(1,dpi=150)
    mpl.rcParams.update({'font.size': 6})
    axMW = plt.subplot(111)
    axMW.plot(x2,y2)
    axMW.set_xlim(3,E0)
    axMW.set_ylim(0,)
    plt.xlabel("Énergie [keV]")
    plt.ylabel("Nombre de photons par [keV·cm²·mGy] @ 1m")
    axMW.grid(which='major', axis='x', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='x', linewidth=0.2, linestyle='-', color='0.85')
    axMW.grid(which='major', axis='y', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='y', linewidth=0.2, linestyle='-', color='0.85')
    axMW.xaxis.set_major_formatter(tic.FormatStrFormatter("%d"))
    axMW.yaxis.set_major_formatter(tic.FormatStrFormatter("%.2g"))
    axMW.xaxis.set_minor_locator(AutoMinorLocator())
    axMW.yaxis.set_minor_locator(AutoMinorLocator())
    axMW.grid(True)
    plt.show()

widgets.interact_manual(spectrum, 
                        E0 = widgets.IntSlider(min=20, max=200, step=10, value=100., description="Haute Tension (kV):", style={'description_width': 'initial'}),
                        H2O_thickness = widgets.IntSlider(min=0, max=500, step=10, value=0., description="Eau (mm):", style={'description_width': 'initial'}));


interactive(children=(IntSlider(value=100, description='Haute Tension (kV):', max=200, min=20, step=10, style=…